In [817]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [818]:
data = pd.read_csv('Data/train.csv')
data_test = pd.read_csv('Data/test.csv')

In [819]:
data.pixels_y = data.pixels_y.astype('int64')

In [820]:
data_test.pixels_y = data_test.pixels_y.fillna(0)
data_test.pixels_y = data_test.pixels_y.astype('int64')

In [821]:
data.dtypes

id                       int64
name                    object
brand                   object
base_name               object
screen_size            float64
pixels_x                 int64
pixels_y                 int64
screen_surface          object
touchscreen              int64
cpu                     object
cpu_details             object
detachable_keyboard    float64
discrete_gpu             int64
gpu                     object
os                      object
os_details              object
ram                      int64
ssd                      int64
storage                  int64
weight                 float64
min_price              float64
max_price              float64
dtype: object

In [822]:
data_s = data[['id','brand', 'screen_size','pixels_y','screen_surface','touchscreen','cpu', "cpu_details", 'discrete_gpu','gpu','os','os_details','ram','ssd','storage','weight','min_price','max_price']]
data_test_s = data_test[['id','brand', 'screen_size','pixels_y','screen_surface','touchscreen','cpu', 'cpu_details', 'discrete_gpu','gpu',"os",'os_details','ram','ssd','storage','weight']]

In [823]:
data_s

,id,brand,screen_size,pixels_y,screen_surface,touchscreen,cpu,cpu_details,discrete_gpu,gpu,os,os_details,ram,ssd,storage,weight,min_price,max_price
0,7774,Lenovo,15.6,1080,Glossy,1,Intel Core i7,Intel Core i7-5500U 2.40 GHz (5th gen Broadwel...,0,Intel HD,Windows,Windows 10,8,0,1000,4.60,899.00,899.00
1,25926,Razer,15.6,1080,Matte,0,Intel Core i7,Intel Core i7-8750H 2.2 GHz (8th gen Coffee La...,1,NVIDIA GeForce RTX 2070 Max-Q,Windows,Windows 10 Home,16,512,512,4.63,2099.99,2099.99
2,25267,HP,15.6,768,NaN,0,AMD A6,AMD A6-9220 2.5 GHz (7th gen Stoney Ridge Dual...,0,AMD Radeon R4,Windows,Windows 10,8,0,500,4.63,439.00,449.00
3,22367,Acer,15.6,1080,Matte,0,Intel Core i3,Intel Core i3-8130U 2.2 GHz (8th gen Kaby Lake...,0,Intel UHD 620,Windows,Windows 10 Home,6,0,1000,5.30,375.00,449.00
4,17471,HP,17.3,900,Glossy,0,Intel Core i5,Intel Core i5-7200U 2.5 GHz (7th gen Kaby Lake...,0,Intel HD 620,Windows,Windows 10,8,0,1000,5.80,559.00,559.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,3897,Dell,15.6,768,Glossy,1,Intel Core i3,Intel Core i3-4030U 1.9 GHz (4th gen Haswell D...,0,Intel HD 4400,Windows,Windows 8.1,4,0,500,6.00,383.51,383.51
506,19301,Asus,17.3,1080,Matte,0,Intel Core i7,Intel Core i7-7700HQ 2.8 GHz (7th gen Kaby Lak...,1,NVIDIA GeForce GTX 1070,Windows,Windows 10 Home,12,128,1128,6.39,1899.99,1899.99
507,22738,Asus,15.6,1080,Matte,0,Intel Core i5,Intel Core i5-8250U 1.6 GHz (8th gen Kaby Lake...,0,Intel UHD 620,Windows,Windows 10 Home,8,128,1128,3.70,499.99,669.97
508,16468,HP,17.3,900,Glossy,0,Intel Core i5,Intel Core i5-7200U 2.5 GHz (7th gen Kaby Lake...,0,Intel HD 620,Windows,Windows 10,8,0,1000,5.84,548.29,548.29


In [824]:
def reformat(df):
    df_pp = df.copy()
    df_pp["cpu_brand"] = df_pp.cpu.str.split(n=1).str[0]
    df_pp["cpu_type"] = df_pp.cpu.str.split(n=1).str[1]
    df_pp["gpu_brand"] = df_pp.gpu.str.split(n=1).str[0]
    df_pp["gpu_series"] = df_pp.gpu.str.split(n=2).str[1]
    df_pp["os_type"] = df_pp.os_details.str.split(n=1).str[1]
    df_pp["cpu_gen"] = df_pp['cpu_details'].str.split("(").str[1].str.split(")").str[0].str.split(" ",n=3).str[0]
    df_pp = df_pp.drop(columns=["os_details", "cpu", "gpu", "os_details"])
    return df_pp

In [825]:
def process_onehot(df, list_values):
    transformed_df = pd.DataFrame()
    from sklearn import preprocessing
    for item in list_values:
        enc = preprocessing.OneHotEncoder()
        df[item] = df[item].astype(str)
        df[item] = df[item].str.lower()
        enc.fit(df[[item]].dropna())
        new_df = pd.DataFrame(data=enc.transform(df[[item]].dropna()).toarray(), columns=enc.categories_[0])
        transformed_df = pd.concat([transformed_df, new_df], axis=1)
    return transformed_df

In [826]:
def process_ordinal(df, list_values):
    transformed_df = pd.DataFrame()
    from sklearn import preprocessing
    for item in list_values:
        new_df = pd.DataFrame()
        enc = preprocessing.OrdinalEncoder()
        df[item] = df[item].astype(str)
        df[item] = df[item].str.lower()
        enc.fit(df[[item]].dropna())
        temp = enc.transform(df[[item]].dropna()).shape
        new_df[item] = enc.transform(df[[item]].dropna()).flatten()
        transformed_df = pd.concat([transformed_df, new_df], axis=1)
    return transformed_df

In [827]:
data_s = reformat(data_s)
data_test_s = reformat(data_test_s)

In [828]:
data_t = pd.concat([data_s, data_test_s]).reset_index(drop=True)
data_t

,id,brand,screen_size,pixels_y,screen_surface,touchscreen,cpu_details,discrete_gpu,os,ram,ssd,storage,weight,min_price,max_price,cpu_brand,cpu_type,gpu_brand,gpu_series,os_type,cpu_gen
0,7774,Lenovo,15.6,1080,Glossy,1,Intel Core i7-5500U 2.40 GHz (5th gen Broadwel...,0,Windows,8,0,1000,4.60,899.00,899.00,Intel,Core i7,Intel,HD,10,5th
1,25926,Razer,15.6,1080,Matte,0,Intel Core i7-8750H 2.2 GHz (8th gen Coffee La...,1,Windows,16,512,512,4.63,2099.99,2099.99,Intel,Core i7,NVIDIA,GeForce,10 Home,8th
2,25267,HP,15.6,768,NaN,0,AMD A6-9220 2.5 GHz (7th gen Stoney Ridge Dual...,0,Windows,8,0,500,4.63,439.00,449.00,AMD,A6,AMD,Radeon,10,7th
3,22367,Acer,15.6,1080,Matte,0,Intel Core i3-8130U 2.2 GHz (8th gen Kaby Lake...,0,Windows,6,0,1000,5.30,375.00,449.00,Intel,Core i3,Intel,UHD,10 Home,8th
4,17471,HP,17.3,900,Glossy,0,Intel Core i5-7200U 2.5 GHz (7th gen Kaby Lake...,0,Windows,8,0,1000,5.80,559.00,559.00,Intel,Core i5,Intel,HD,10,7th
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,8398,Asus,13.3,1080,Matte,0,Intel Core M-5Y10 0.8 GHz (5th gen Broadwell D...,0,Windows,8,256,256,2.65,NaN,NaN,Intel,Core m3,Intel,HD,10,5th
728,21211,LG,15.6,1080,Glossy,1,Intel Core i7-8550U 1.8 GHz (8th gen Kaby Lake...,0,Windows,16,1000,1000,2.41,NaN,NaN,Intel,Core i7,Intel,UHD,10 Home,8th
729,22553,Razer,15.6,1080,Matte,0,Intel Core i7-8750H 2.2 GHz (8th gen Coffee La...,1,Windows,16,512,512,4.63,NaN,NaN,Intel,Core i7,NVIDIA,GeForce,10 Home,8th
730,28808,MSI,15.6,1080,Matte,0,Intel Core i5-8300H 2.3 GHz (8th gen Coffee La...,1,Windows,8,256,256,4.10,NaN,NaN,Intel,Core i5,NVIDIA,GeForce,10 Home,8th


In [829]:
#processed_data_total = process_onehot(data_t,['screen_surface', 'cpu_type', 'gpu_brand', "gpu_series", 'os_type','brand', "cpu_gen","os"  ])

In [830]:
processed_data_total = process_ordinal(data_t,['screen_surface', 'cpu_type', 'gpu_brand', "gpu_series", 'os_type','brand', "cpu_gen","os" ])

In [831]:
#data_f = pd.concat([processed_data,data_s[['id','pixels_y','ram','ssd','storage','weight', 'discrete_gpu','touchscreen','min_price','max_price']]], axis=1).set_index('id')
#data_f_test = pd.concat([processed_data_test,data_test_s[['id','pixels_y','ram','ssd','storage','weight','touchscreen', 'discrete_gpu']]], axis=1).set_index('id').fillna(0)

In [832]:
data_t = data_t[['id','pixels_y','ram','ssd','storage','weight', 'discrete_gpu','touchscreen','min_price','max_price']]

In [833]:
processed_data_total

,screen_surface,cpu_type,gpu_brand,gpu_series,os_type,brand,cpu_gen,os
0,0.0,10.0,2.0,3.0,0.0,9.0,4.0,4.0
1,1.0,10.0,7.0,1.0,1.0,14.0,7.0,4.0
2,2.0,3.0,0.0,9.0,0.0,6.0,6.0,4.0
3,1.0,8.0,2.0,11.0,1.0,0.0,7.0,4.0
4,0.0,9.0,2.0,3.0,0.0,6.0,6.0,4.0
...,...,...,...,...,...,...,...,...
727,1.0,13.0,2.0,3.0,0.0,3.0,4.0,4.0
728,0.0,10.0,2.0,11.0,1.0,10.0,7.0,4.0
729,1.0,10.0,7.0,1.0,1.0,14.0,7.0,4.0
730,1.0,9.0,7.0,1.0,1.0,12.0,7.0,4.0


In [834]:
data_t.dtypes

id                int64
pixels_y          int64
ram               int64
ssd               int64
storage           int64
weight          float64
discrete_gpu      int64
touchscreen       int64
min_price       float64
max_price       float64
dtype: object

In [835]:
data_total = pd.concat([processed_data_total, data_t], axis=1).set_index("id")

In [836]:
data_total

,screen_surface,cpu_type,gpu_brand,gpu_series,os_type,brand,cpu_gen,os,pixels_y,ram,ssd,storage,weight,discrete_gpu,touchscreen,min_price,max_price
id,,,,,,,,,,,,,,,,,
7774,0.0,10.0,2.0,3.0,0.0,9.0,4.0,4.0,1080,8,0,1000,4.60,0,1,899.00,899.00
25926,1.0,10.0,7.0,1.0,1.0,14.0,7.0,4.0,1080,16,512,512,4.63,1,0,2099.99,2099.99
25267,2.0,3.0,0.0,9.0,0.0,6.0,6.0,4.0,768,8,0,500,4.63,0,0,439.00,449.00
22367,1.0,8.0,2.0,11.0,1.0,0.0,7.0,4.0,1080,6,0,1000,5.30,0,0,375.00,449.00
17471,0.0,9.0,2.0,3.0,0.0,6.0,6.0,4.0,900,8,0,1000,5.80,0,0,559.00,559.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8398,1.0,13.0,2.0,3.0,0.0,3.0,4.0,4.0,1080,8,256,256,2.65,0,0,NaN,NaN
21211,0.0,10.0,2.0,11.0,1.0,10.0,7.0,4.0,1080,16,1000,1000,2.41,0,1,NaN,NaN
22553,1.0,10.0,7.0,1.0,1.0,14.0,7.0,4.0,1080,16,512,512,4.63,1,0,NaN,NaN


In [912]:
data_total_train = data_total.filter(items=data.id, axis=0)
data_total_test = data_total.filter(items=data_test.id, axis=0).iloc[:,0:-2].fillna(0)
index=data_total_test.index

In [913]:
data_total_test

,screen_surface,cpu_type,gpu_brand,gpu_series,os_type,brand,cpu_gen,os,pixels_y,ram,ssd,storage,weight,discrete_gpu,touchscreen
id,,,,,,,,,,,,,,,
28807,0.0,10.0,2.0,11.0,4.0,3.0,7.0,4.0,1080,16,512,512,2.42,0,1
22559,1.0,8.0,2.0,3.0,1.0,4.0,6.0,4.0,768,8,0,1000,4.95,0,0
28647,1.0,10.0,2.0,11.0,1.0,3.0,7.0,4.0,1080,8,256,1256,4.00,0,0
22141,0.0,9.0,2.0,3.0,1.0,6.0,6.0,4.0,768,8,128,2128,4.52,0,1
26116,1.0,10.0,7.0,1.0,1.0,12.0,7.0,4.0,1080,32,512,512,4.96,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8398,1.0,13.0,2.0,3.0,0.0,3.0,4.0,4.0,1080,8,256,256,2.65,0,0
21211,0.0,10.0,2.0,11.0,1.0,10.0,7.0,4.0,1080,16,1000,1000,2.41,0,1
22553,1.0,10.0,7.0,1.0,1.0,14.0,7.0,4.0,1080,16,512,512,4.63,1,0


In [914]:
from sklearn import decomposition
from sklearn import model_selection
from sklearn import ensemble
from sklearn import cluster
from sklearn import svm
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.model_selection import LeavePOut
from sklearn.ensemble import BaggingRegressor

In [915]:

X_1 = data_total_train.iloc[:,0:-2].fillna(0)
y_1 = data_total_train.iloc[:,-2].fillna(0)
X_2 = data_total_train.iloc[:,0:-2].fillna(0)
y_2 = data_total_train.iloc[:,-1].fillna(0)
y_2

id
7774      899.00
25926    2099.99
25267     449.00
22367     449.00
17471     559.00
          ...   
3897      383.51
19301    1899.99
22738     669.97
16468     548.29
29633    1699.00
Name: max_price, Length: 510, dtype: float64

In [916]:
clustering = cluster.FeatureAgglomeration(n_clusters=50)
#X_1 = clustering.fit_transform(X_1)
#X_2 = clustering.fit_transform(X_2)
#data_total_test = clustering.fit_transform(data_total_test)

In [917]:
data_total_test.shape

(222, 15)

In [918]:
#pca = decomposition.PCA(n_components=20)
#X_1 = pca.fit_transform(X_1)
#X_2 = pca.fit_transform(X_2)
#data_f_test_pp = pca.fit_transform(data_f_test_pp)

In [919]:
pp = preprocessing.PowerTransformer(standardize=True)
#X_1 = pp.fit_transform(X_1)
#X_2 = pp.fit_transform(X_2)
#data_total_test = pp.fit_transform(data_total_test)


In [960]:
parameters = {'n_estimators':np.linspace(50,300,5).astype(int), 'criterion':['mae']}
#model = ensemble.RandomForestRegressor(n_estimators=200)
#model = BaggingRegressor(base_estimator=ensemble.RandomForestRegressor(n_estimators=200),n_estimators=10,random_state=40)
#model = BaggingRegressor(base_estimator=ensemble.RandomForestRegressor(n_estimators=200),n_estimators=10,random_state=40)
model = ensemble.GradientBoostingRegressor(n_estimators=5000)
model2 = ensemble.RandomForestRegressor(n_estimators=5000)
model3 = ensemble.RandomForestRegressor(n_estimators=5000)
model4 = ensemble.GradientBoostingRegressor(n_estimators=5000)
lpout = LeavePOut(50)

#clf = model_selection.GridSearchCV(model, parameters, cv=5, scoring='neg_median_absolute_error', n_jobs=6)
#clf2 = model_selection.GridSearchCV(model2, parameters, cv=5, scoring='neg_median_absolute_error', n_jobs=6)

In [961]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_1,y_1, test_size=0.3, random_state=40 )
predictions = pd.DataFrame(index=y_test.index)

#clf.fit(X_train, y_train)
#model = clf.best_estimator_
model.fit(X_train, y_train)
predictions['min_pred'] = model.predict(X_test)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X_2,y_2, test_size=0.3, random_state=40)
#clf.fit(X_train, y_train)
#model2 = clf.best_estimator_
model2.fit(X_train, y_train)
predictions['max_pred'] = model2.predict(X_test)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X_1,y_1, test_size=0.3, random_state=40 )
#clf.fit(X_train, y_train)
#model = clf.best_estimator_
model3.fit(X_train, y_train)
predictions['min_pred2'] = model3.predict(X_test)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X_2,y_2, test_size=0.3, random_state=40)
#clf.fit(X_train, y_train)
#model2 = clf.best_estimator_
model4.fit(X_train, y_train)
predictions['max_pred2'] = model4.predict(X_test)

In [962]:
predictions

,min_pred,max_pred,min_pred2,max_pred2
id,,,,
22739,1080.550216,1007.931242,976.877772,1376.312843
27244,1628.999498,1520.630240,1545.003498,1507.327611
24846,358.133815,443.759808,425.737985,350.438080
19617,427.966148,389.970184,365.863216,435.843105
29410,199.969438,220.139962,209.588025,214.995535
...,...,...,...,...
22860,82.362493,263.798024,241.827954,124.062638
10371,736.244985,644.578448,640.782716,812.958619
3841,444.353570,362.437007,349.696576,424.951070


In [971]:
def finetune_predictions_test(results):
    for i,prediction in enumerate(results[["min_pred","max_pred"]].index):
        results.loc[prediction]["min_pred"] = min(results.loc[prediction][["min_pred","max_pred"]])
    return results



In [963]:
def finetune_predictions(results):
    for i,prediction in enumerate(results[["min_pred","max_pred"]].index):
        results.loc[prediction]["min_pred"] = np.mean(results.loc[prediction][["min_pred", "min_pred2"]])
        results.loc[prediction]["max_pred"] = np.mean(results.loc[prediction][["max_pred", "max_pred2"]])
        results.loc[prediction]["min_pred"] = min(results.loc[prediction][["min_pred","max_pred"]])
    return results

In [964]:
predictions = finetune_predictions(predictions)
predictions['min_true'] = y_test
predictions['min_error'] = np.abs(predictions.min_pred-predictions.min_true)
predictions['max_true'] = y_test
predictions['max_error'] = np.abs(predictions.max_pred-predictions.max_true)

In [965]:
print(predictions.min_error.mean())
print(predictions.max_error.mean())

158.35158898218216
176.07844037205248


In [966]:
np.mean(predictions.min_error+predictions.max_error)


334.43002935423465

In [967]:
predictions

,min_pred,max_pred,min_pred2,max_pred2,min_true,min_error,max_true,max_error
id,,,,,,,,
22739,1028.713994,1192.122043,976.877772,1376.312843,759.99,268.723994,759.99,432.132043
27244,1513.978926,1513.978926,1545.003498,1507.327611,1999.11,485.131074,1999.11,485.131074
24846,391.935900,397.098944,425.737985,350.438080,378.98,12.955900,378.98,18.118944
19617,396.914682,412.906645,365.863216,435.843105,648.00,251.085318,648.00,235.093355
29410,204.778731,217.567748,209.588025,214.995535,262.98,58.201269,262.98,45.412252
...,...,...,...,...,...,...,...,...
22860,162.095224,193.930331,241.827954,124.062638,198.49,36.394776,198.49,4.559669
10371,688.513851,728.768533,640.782716,812.958619,549.00,139.513851,549.00,179.768533
3841,393.694038,393.694038,349.696576,424.951070,349.00,44.694038,349.00,44.694038


In [968]:
predictions.merge(data.set_index('id'), left_index=True, right_index=True).merge(data_total_train, left_index=True, right_index=True).sort_values('min_error', ascending=False).to_excel("steven_output.xlsx")

In [969]:
data_total_test.shape

(222, 15)

In [973]:
predictions = pd.DataFrame(index=index)
model.fit(X_1,y_1)
model2.fit(X_2,y_2)
model3.fit(X_1,y_1)
model4.fit(X_2,y_2)
predictions['min_pred'] = model.predict(data_total_test)
predictions['max_pred'] = model2.predict(data_total_test)
predictions['min_pred2'] = model3.predict(data_total_test)
predictions['max_pred2'] = model4.predict(data_total_test)
finetune_predictions(predictions)

,min_pred,max_pred,min_pred2,max_pred2
id,,,,
28807,1082.508600,1160.825813,1086.245494,1142.906353
22559,374.708025,399.331544,369.718927,409.871907
28647,758.442623,787.999577,723.420906,824.248325
22141,552.095377,560.760792,547.966714,560.031726
26116,1268.432983,1390.049630,1304.748113,1416.544811
...,...,...,...,...
8398,1018.547752,1086.206919,983.469594,1146.702707
21211,1400.285238,1421.661849,1407.539171,1357.591785
22553,2189.390485,2288.526160,2178.352396,2301.909497


In [979]:
predictions.iloc[:,0:2].to_csv('results_steven.csv')

In [980]:
predictions.iloc[:,0:2]

,min_pred,max_pred
id,,
28807,1082.508600,1160.825813
22559,374.708025,399.331544
28647,758.442623,787.999577
22141,552.095377,560.760792
26116,1268.432983,1390.049630
...,...,...
8398,1018.547752,1086.206919
21211,1400.285238,1421.661849
22553,2189.390485,2288.526160


In [ ]:
data_total_test.columns

In [ ]:
X_1.columns